# List Sessions

This notebook lists all sessions or sessions for a specific user.

**Use this to:**
- View all sessions in the database
- Find session IDs for deletion or update operations
- Filter sessions by user_id


In [ ]:
import sys
from pathlib import Path
from typing import Optional

# Add project root to path
project_root = Path().resolve().parent.parent
sys.path.insert(0, str(project_root))

from src.adapters.database.database_handler import DatabaseHandler


## Configuration

Set `USE_DYNAMODB = True` to use DynamoDB, or `False` to use CSV files.


In [ ]:
# Configuration
USE_DYNAMODB = True  # Set to False to use CSV instead


In [ ]:
def list_sessions(user_id: Optional[str] = None, db_write_allowed: bool = True) -> None:
    """
    List all sessions or sessions for a specific user.
    
    Args:
        user_id: Optional user ID to filter by. If None, lists all sessions.
        db_write_allowed: If True, use DynamoDB; if False, use CSV
    """
    print("=" * 60)
    print("Listing Sessions")
    print("=" * 60)
    if user_id:
        print(f"Filter: user_id = {user_id}")
    else:
        print("Filter: All sessions")
    print("=" * 60)
    
    db_handler = DatabaseHandler(db_write_allowed=db_write_allowed)
    
    try:
        session_responses = db_handler.load_table("session_responses")
        
        if session_responses.empty:
            print("[INFO] No sessions found")
            db_handler.close()
            return
        
        # Filter by user_id if provided
        if user_id:
            filtered = session_responses[session_responses["user_id"] == user_id]
        else:
            filtered = session_responses
        
        if filtered.empty:
            print(f"[INFO] No sessions found for user_id={user_id}")
            db_handler.close()
            return
        
        print(f"\n[OK] Found {len(filtered)} session(s):\n")
        
        # Display sessions
        for idx, (_, row) in enumerate(filtered.iterrows(), 1):
            print(f"Session {idx}:")
            print(f"  Session ID: {row.get('id')}")
            print(f"  User ID: {row.get('user_id')}")
            print(f"  Name: {row.get('name')}")
            print(f"  Boyfriend Name: {row.get('boyfriend_name')}")
            print(f"  Toxic Score: {row.get('toxic_score')}")
            print(f"  Filter Violations: {row.get('filter_violations')}")
            print(f"  Language: {row.get('language')}")
            print(f"  Session Start: {row.get('session_start_time')}")
            print()
        
        db_handler.close()
        
    except Exception as e:
        print(f"\n[ERROR] Error listing sessions: {e}")
        import traceback
        print(f"[ERROR] Traceback: {traceback.format_exc()}")
        db_handler.close()


In [ ]:
# Example: List all sessions
list_sessions(db_write_allowed=USE_DYNAMODB)


In [ ]:
# Example: List sessions for a specific user
# Uncomment and modify the user_id below

# list_sessions(user_id="your_user_id_here", db_write_allowed=USE_DYNAMODB)
